## Big 5
Et datasett jeg fant. Se på korrelasjoner og vise de i Jupyter notebook
https://openpsychometrics.org/_rawdata/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
df_data = pd.read_csv("b5_svar.csv")
print(df_data.shape)
df_data.head

In [ ]:
# Plotte de fem gjennomsnittskolonnene fordelt på kjønn
spm_grupper = {"E": "Extraversion", "A": "Agreeableness", "C": "Conscientiousness", "N": "Emotional Stability", "O": "Intellect/Imagination"}
five = ['N', 'C', 'E', 'A', 'O']
df_plot = df_data[five]
pd.plotting.scatter_matrix(df_plot, alpha = 0.07, figsize = (12.0,12.0))
plt.savefig("scattermatrix.jpg")

In [ ]:
fig=plt.figure(figsize = (18,12))
bins = np.linspace(0, 5, 11)
for i, kat in enumerate(five):
    ax=fig.add_subplot(2,3,i+1)
    x = df_data[kat][df_data["gender"] == 1]
    y = df_data[kat][df_data["gender"] == 2]
    print("\n"+kat, spm_grupper[kat])
    plt.hist([x, y], bins, label=['male', 'female'], density = True)
    plt.legend(loc='upper right')
    plt.title(kat+" "+spm_grupper[kat])

plt.savefig("kat_histogram.jpg")
plt.show()




In [ ]:
# Få histogram på kvinner og menn, med alder bortover
bins = np.linspace(0, 80, 41)
plt.figure(figsize=[10,8])
x = df_data["age"][df_data["gender"] == 1]
y = df_data["age"][df_data["gender"] == 2]
plt.hist([x, y], bins,label=['male', 'female'])
plt.show()
print("Age,  male,  female")
for age, frame in df_data.groupby("age"):
    print(age, len(frame[frame["gender"]== 1]), len(frame[frame["gender"]== 2]))

In [ ]:
# Histogram på landskodene
# Lage bar plot med landene som søyler, og gjennomsnitt for landet og bokstaven som verdi.

# Iterere gjennom 
df_data.country.value_counts()[:10]
cc5 = ['US', 'GB', 'IN', 'AU', 'CA']

for i, kat in enumerate(five):
    df_c = df_data[[kat,"country"]]
    df_c["cc_5"] = df_c["country"].where(df_c["country"].isin(cc5), "other")
    df_c.rename({kat:kat+"_"+spm_grupper[kat]}, axis=1, inplace = True)
    df_c.groupby("cc_5").mean().plot(kind='bar', ylim=(2.5,4) , color='brown')
    plt.savefig("country_"+kat+".jpg")




## Utvikling over tid 
Nå skal vi lage linjediagram der vi tar gjennomsnittet for hver av de fem for hvert årstall, og så hver årsgruppe

In [ ]:
fig=plt.figure(figsize = (18,12))
'''
Hvilke ulike plott? FOr hver i five.
Lage filter på dette, og groupby på kjønn'''
df_gender_age = df_data[df_data["gender"].isin([1, 2]) &  df_data.age.isin(range(60))]# Exclude 0 and 3
df_gender_age["gender"] = df_gender_age["gender"].map({1:"Male", 2:"Female"})
for c in five:
    dfc = df_gender_age[["gender", "age"]+[c]].rename(spm_grupper, axis = 1)
    print(dfc)
    dfc.groupby(["gender", "age"]).mean().unstack("gender").plot(label="Hei")
    # df_data[m_f][["gender", "age"]+five].groupby(["gender", "age"]).mean()
    # df_gender_age[df_gender_age["gender"] == g][["gender", "age"]+["O"]].groupby(["gender", "age"]).count()

    

In [ ]:
## Nå med rolling average
# For hver kategori: Lage en DataFrame med gender, c, age og gjennomsnittet
# Groupby på gender, og passe på at den er sortert og så lage rolling average

''' 
Hvilke ulike plott? FOr hver i five.
Lage filter på dette, og groupby på kjønn '''

dfga = df_data[df_data["gender"].isin([1, 2]) &  df_data.age.isin(range(16,51))]# Exclude 0 and 3
def feiler():
    for c in five:
        dfgc = dfga[["gender", "age"]+[c]]
        dfgc["5yr_avg"] = dfgc[["gender", "age"]+[c]].groupby(["gender", "age"]).rolling(5,1)[c].mean().reset_index(0,drop=True)
        dfgc = dfgc.drop(c)
        dfgc.groupby(["gender", "age"]).mean().unstack("gender").plot()

    # print(df_gender_age[["gender", "age"]+[c]].groupby(["gender", "age"]).mean().unstack("gender").columns)
# df["R"]  = df.groupby("X").rolling(2, min_periods=1)["B"].mean().reset_index(drop=True)


df_agg = pd.DataFrame() # Bygger den opp for læringen
for c in five:
    dfgc = dfga[["gender", "age"]+[c]]
    for age, frame in dfgc.groupby(["age"]):
        m = frame[frame["gender"] == 1].agg({c:"mean"})[c]
        f = frame[frame["gender"] == 2].agg({c:"mean"})[c]
        df_agg = df_agg.append({"category":c,"age": age ,"male": m, "female":f }, ignore_index=True)

nrows, ncols = 3,2

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize = (20,20))
for idx, c in enumerate(five):
    df_plot= df_agg[df_agg["category"]== c ][["age", "male", "female"]].set_index("age").sort_index()
    df_plot["male_5_avg"] = df_plot["male"].rolling(5, min_periods=1).mean()
    df_plot["female_5_avg"] = df_plot["female"].rolling(5, min_periods=1).mean()
    print(c)
    df_plot[[ "male_5_avg", "female_5_avg"]].plot(title = c+"-"+spm_grupper[c], ax = axes[idx // ncols, idx % ncols])
fig.savefig("kat_alder.jpg")
fig.show()

In [ ]:
df_agg

In [ ]:
print(df.columns)
df.index

## Importere data
Jeg må ta inn informasjon om hvert spørsmål. Vite hvilken kategori de hører til, hva den vil si, og hvilke spørsmål som skal snus
Tenker jeg lager det i excel, og så tar det inn her etterpå

In [ ]:
# Bare kjøre denne hvis vi skal lage ny b5_svar.csv
df = pd.read_csv("data.csv", delimiter = "\t")
spm_grupper = {"1": "Extraversion", "2": "Agreeableness", "3": "Conscientiousness", "4": "Emotional Stability", "5": "Intellect/Imagination"}

def likerettet (svarverdi, fortegn):
    """ Tar inn svarverdien og snur den hvis fortegn på spørsmålesskalaen er -
    5 blir 1, 2 blir 4 , 1 blir 5
    """
    if fortegn == '-':
        return 6-svarverdi
    return svarverdi

df_c = pd.read_csv ("konvertering1.csv", sep = ";") 
print(df_c.columns)
df_c["Kat"] = df_c["Kat-Retning"].str[0]
df_c["Retning"] = df_c["Kat-Retning"].str[1]
df_c=df_c.set_index('DS_kode')
print(df_c.at['E4','Retning'])

# Mappe funksjonen mot resultatet

df_svar = df[['race', 'age', 'engnat', 'gender', 'hand', 'source', 'country']].copy()
spms = ['E1',
       'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'N1', 'N2', 'N3',
       'N4', 'N5', 'N6', 'N7', 'N8', 'N9', 'N10', 'A1', 'A2', 'A3', 'A4', 'A5',
       'A6', 'A7', 'A8', 'A9', 'A10', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7',
       'C8', 'C9', 'C10', 'O1', 'O2', 'O3', 'O4', 'O5', 'O6', 'O7', 'O8', 'O9',
       'O10']
for spm in  spms:
       for idx in df_svar.index:
           df_svar.at[idx,spm] = likerettet(df.at[idx, spm], df_c.at[spm,'Retning'])

print("df og df_svar ",df.shape, df_svar.shape)

for kat in ['E', 'N','A', 'C', 'O' ]: #Gjennomsnitt for hver kategori
    c_idxs = [kat + str(n) for n in range(1,11)]
    df_svar[kat] = df_svar[c_idxs].mean(axis=1)
df_svar.to_csv("b5_svar.csv")

In [ ]:
## Tester importrutinen
### likerettet()
for n in range(1,6):
    assert likerettet(n, '+') == n
    assert likerettet(n, '-') == 6-n
assert likerettet(5, '-') == 1

print('OK')

### Kladd

df_test = pd.DataFrame({'i':['a', 'b', 'c'],'en':[1,11,111], 'to':[2,22,222]})
df_test = df_test.set_index('i')
for idx in df_test.index:
    for c_idx in ['en', 'to']:
        print(likerettet(df_test.at[idx,c_idx], df_c.at['E4','Retning']))
df_test['g'] = df_test[['en','to']].mean(axis=1)
df_test



In [ ]:
df = pd.DataFrame({'B': [0, 1,3,4, 2, np.nan, 4], "X": [2,2,2,3,3,3,3]}).sort_values(["X", "B"])
df["R"]  = df.groupby("X").rolling(2, min_periods=1)["B"].mean().reset_index(drop=True)
df["Y"] = df["X"].where(df["X"].isin([1,2]) , 100 )
df